## continuumio/miniconda3


The continuumio/miniconda3 in the Dockerfile is the base Docker image that's being used to build the custom Docker image. It's the starting point for the Docker image.

## WORKDIR /app

In a Dockerfile, the WORKDIR instruction is used to change the working directory of a Docker container for any following operations in the Dockerfile (RUN, CMD, ENTRYPOINT, COPY and ADD instructions). If the WORKDIR doesn’t exist, it will be created even if it’s not used in any subsequent Dockerfile instruction.

So, WORKDIR /app means that the Docker container changes its working directory (the directory you're in when you start a command line session, or in this case, where Docker will execute subsequent commands) to /app. This /app directory is inside the Docker container, not on your host machine.

If, for example, you have the following command later in your Dockerfile: COPY . ., this would copy files from your Docker context (the directory and its subdirectories on your machine where the Dockerfile is located) into the /app directory in the Docker container, because you've set that as the working directory.

It's a good practice to set WORKDIR to a dedicated location in your Docker container for your application to keep your environment tidy and predictable.

In [ ]:
sos run xqtl-pipeline/code/misc/docker_build-Copy1.ipynb dcontainer_generator\
    --container_list xqtl-pipeline/code/misc/xQTL_conda_packages1.csv \
    --env bioinfo fastenloc leafcutter METAL methylation polyfun psichomics rna_quantification stephenslab TensorQTL

In [ ]:
[global]
parameter: container_list = path
parameter: env = list
parameter: output = env 
parameter: cwd=path("output6")

In [ ]:
[dcontainer_generator_1]
input: container_list, for_each="env"
output: f'{cwd}/{_env}/Dockerfile.{_env}'
python: expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout'
    import csv
    from collections import defaultdict

    def write_dockerfile(env, packages):
        header = """\
    # Use miniconda as base image
    FROM continuumio/miniconda3

    # Set the working directory
    WORKDIR /app

    # Updating conda
    RUN conda update -n base -c defaults conda
    """
        with open(f'$[_output]', 'w') as f:
            content = header + "\n" + "\n".join(packages)
            f.write(content)

    # Use defaultdict to automatically create a new list when a new key is encountered
    packages_by_env = defaultdict(list)
    packages_names_by_env = defaultdict(set)
    universial_packages = []

    with open('$[_input]', 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            if not row['Conda package']: 
                # Skip the current row if the 'Conda Package' field is empty
                continue
            # Parse environments (assuming they are comma-separated)
            environments = row['Environment'].split(',')
            for env in environments:
                # Strip spaces and lower-case the environment name for comparison
                env = env.strip().lower()
                install_cmd = f"RUN conda install -c {row['Channel']} {row['Conda package'].split('/')[-1]}{'=' if len(row['Version']) != 0 or len(row['CondaVersion']) != 0 else f''}{row['CondaVersion'] if len(row['CondaVersion']) != 0 else row['Version']} -y"

                # If the package is universial, add it to the universial_packages list
                if env == 'universial':
                    universial_packages.append(install_cmd)
                else:
                    # Add the installation command to the corresponding environment
                    packages_by_env[env].append(install_cmd)
                    packages_names_by_env[env].add(row['Conda package'].split('/')[-1])

    # Write a Dockerfile for the environment only, including universal packages
    env = '$[_env]'.lower()
    packages = packages_by_env[env]

    # Filter out universal packages that have been specifically requested by this environment
    filtered_universial_packages = [cmd for cmd in universial_packages if cmd.split()[5].split('=')[0] not in packages_names_by_env[env]]

    # Include universal packages in the environment
    all_packages = filtered_universial_packages + packages

    write_dockerfile(env, all_packages)

In [ ]:
[dcontainer_generator_2]
output: f'{_input}.tar'
bash: expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout'
    cd $[_input:d]
    export DOCKER_BUILDKIT=1 #need docker vesion greater than 18.09 to change default value from 0 to 1 if it's greater than 23.0 then the default value is 1 already
    docker build -t my-conda-env-[env] -f $[_input] .
    docker save my-conda-env-[env] > $[_output:b].tar
    # to use the container you can 'docker load' the tar file